In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import FinanceDataReader as fdr
import csv

In [2]:
import requests
import requests as req
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time

In [3]:
from html_table_parser import parser_functions
from html_table_parser import parser_functions as parser

### 종목 이름과 코드 긁어오기

In [6]:
name_code = {}

for i in range(1, 38):
    driver = webdriver.Chrome()
    driver.get(f'https://kr.investing.com/stock-screener/?sp=country::11|sector::a|industry::a|equityType::a|exchange::a|revtrendgr_us::-60.39,238.9%3Ceq_market_cap;{i}')
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    temp = soup.find_all('table')
    t = parser.make2d(temp[11])
    print(i)
    
    for j in range(1, len(t)):
        name_code[t[j][1]] = t[j][2]
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


In [7]:
name_code

{'삼성전자우': '005935',
 '삼성전자': '005930',
 'SK하이닉스': '000660',
 'LG 화학': '051915',
 '삼성바이오로직스': '207940',
 '현대차2우B': '005387',
 '현대차우': '005385',
 '현대차3우B': '005389',
 '삼성SDI': '006400',
 '삼성SDI우': '006405',
 'NAVER': '035420',
 'LG화학': '051910',
 '현대차': '005380',
 '카카오': '035720',
 '기아': '000270',
 '삼성물산우B': '02826K',
 'LG전자우': '066575',
 'LG생활건강우': '051905',
 'SK이노우': '096775',
 'KB금융': '105560',
 '신한지주': '055550',
 'POSCO홀딩스': '005490',
 'LG전자': '066570',
 'SK우': '03473K',
 '삼성물산': '028260',
 '현대모비스': '012330',
 'SK이노베이션': '096770',
 'LG우': '003555',
 '한국전력': '015760',
 '삼성전기우': '009155',
 'SK그룹': '034730',
 'HMM': '011200',
 '하나금융지주': '086790',
 '아모레퍼시픽우': '090435',
 '두산중공업': '034020',
 'LG생활건강': '051900',
 'LG': '003550',
 'S-oil': '010950',
 'SK텔레콤': '017670',
 '삼성전기': '009150',
 '삼성생명': '032830',
 '삼성에스디에스': '018260',
 'S-Oil우': '010955',
 '우리금융지주': '316140',
 'Charmzone Global': '158310',
 '삼성화재 우선주': '000815',
 '대한항공': '003490',
 '아모레퍼시픽': '090430',
 '대한항공우': '003495',
 '포스코케미칼':

In [8]:
len(name_code)

1835

In [96]:
print(list(name_code.values())[279])

290650


In [77]:
name_code = pd.read_csv('종목 및 코드.csv')
name_code = name_code.drop(['Unnamed: 0'], axis = 1)
name_code

,name,code
0,삼성전자우,005935
1,삼성전자,005930
2,SK하이닉스,000660
3,LG 화학,051915
4,삼성바이오로직스,207940
...,...,...
1722,EMNI,083470
1723,지나인제약,078650
1724,에스디시스템,121890
1725,SNK,950180


In [79]:
len(name_code['name'].unique())

1727

In [80]:
len(name_code['code'].unique())

1725

### Selenium 재무제표

In [108]:
df_oo = []
cnt = 0


for i,j in name_code.items():
    try:
        driver = webdriver.Chrome()
        driver.get(f'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={j}')
        driver.find_element_by_css_selector('#cns_Tab21').click()
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        temp = soup.find_all('table')
        p = parser.make2d(temp[12])
        globals()['df_'+str(j)]= pd.DataFrame(p[2:], columns = p[1])
        globals()['df_'+str(j)]=globals()['df_'+str(j)].set_index('주요재무정보').T
        globals()['df_'+str(j)]['name'] = i
        globals()['df_'+str(j)]['code'] = j
        globals()['df_'+str(j)]['year'] = [2017, 2018, 2019, 2020, 2021, 0, 0, 0]   
        globals()['df_'+str(j)]=globals()['df_'+str(j)].drop(globals()['df_'+str(j)].index[5:])


        df_oo.append(globals()['df_'+str(j)])

        cnt += 1
        print(cnt)
    
    except:
        pass

df_financial_statements = pd.concat(df_oo)     

C:\Users\USER\AppData\Local\Temp/ipykernel_14152/441072917.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#cns_Tab21').click()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [109]:
df_financial_statements

주요재무정보,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,...,PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주),name,code,year
2017/12\n\t\t\t\t\t\t\t\t\t\n(IFRS연결),"2,395,754","536,450","536,450","561,960","421,867","413,446","8,422","3,017,521","872,607","2,144,914",...,9.40,"30,427",1.67,850,1.67,14.09,"6,454,924,700",삼성전자우,005935,2017
2018/12\n\t\t\t\t\t\t\t\t\t\n(IFRS연결),"2,437,714","588,867","588,867","611,600","443,449","438,909","4,540","3,393,572","916,041","2,477,532",...,6.42,"35,342",1.09,"1,416",3.66,21.92,"5,969,782,550",삼성전자우,005935,2018
2019/12\n\t\t\t\t\t\t\t\t\t\n(IFRS연결),"2,304,009","277,685","277,685","304,322","217,389","215,051","2,338","3,525,645","896,841","2,628,804",...,17.63,"37,528",1.49,"1,416",2.54,44.73,"5,969,782,550",삼성전자우,005935,2019
2020/12\n\t\t\t\t\t\t\t\t\t\n(IFRS연결),"2,368,070","359,939","359,939","363,451","264,078","260,908","3,170","3,782,357","1,022,877","2,759,480",...,21.09,"39,406",2.06,"2,994",3.70,77.95,"5,969,782,550",삼성전자우,005935,2020
2021/12\n\t\t\t\t\t\t\t\t\t\n(IFRS연결),"2,796,048","516,339","516,339","533,518","399,074","392,438","6,637","4,266,212","1,217,212","3,048,999",...,13.55,"43,611",1.80,"1,444",1.84,25.00,"5,969,782,550",삼성전자우,005935,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017/12\n\t\t\t\t\t\t\t\t\t\n(IFRS연결),"1,285",-31,-31,7,3,4,-1,803,417,386,...,168.16,"13,851",1.97,0,0.00,0.00,"2,811,871",Hankook,050540,2017
2018/12\n\t\t\t\t\t\t\t\t\t\n(IFRS연결),"1,284",-93,-93,-187,-193,-188,-5,825,402,423,...,N/A,"11,018",2.77,0,0.00,0.00,"3,878,559",Hankook,050540,2018
2019/12\n\t\t\t\t\t\t\t\t\t\n(IFRS연결),"1,211",-123,-123,-420,-370,-352,-18,899,779,120,...,N/A,"2,978",6.83,0,0.00,0.00,"3,878,559",Hankook,050540,2019
2020/12\n\t\t\t\t\t\t\t\t\t\n(IFRS연결),"1,107",-7,-7,-52,-45,-40,-5,565,477,88,...,N/A,"1,954",1.44,0,0.00,0.00,"5,078,559",Hankook,050540,2020


In [110]:
df_financial_statements.to_csv('재무제표.csv')

In [6]:
len(pd.read_csv('재무제표.csv')['name'].unique())

1727

### 재무제표 TEST

In [98]:
li = ['074600', '041140','001680', '290650']

In [106]:
for i in li:
    try:
        driver = webdriver.Chrome()
        driver.get(f'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={i}')
        driver.find_element_by_css_selector('#cns_Tab21').click()
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        temp = soup.find_all('table')
        p = parser.make2d(temp[12])
        globals()['df_'+str(i)]= pd.DataFrame(p[2:], columns = p[1])
        globals()['df_'+str(i)]=globals()['df_'+str(i)].set_index('주요재무정보').T
        globals()['df_'+str(i)]['name'] = 'pp'
        globals()['df_'+str(i)]['code'] = i
        globals()['df_'+str(i)]['year'] = [2017, 2018, 2019, 2020, 2021, 0, 0, 0]
        globals()['df_'+str(i)]=globals()['df_'+str(i)].drop(globals()['df_'+str(i)].index[5:])
#         df= pd.DataFrame(p[2:], columns = p[1])
#         df=df.set_index('주요재무정보').T
#         df['name'] = '잉'
#         df['code'] = '001680'
#         df['year'] = [2017, 2018, 2019, 2020, 2021, 0, 0, 0]
#         df = df.drop(df.index[5:])
    except:
        pass

C:\Users\USER\AppData\Local\Temp/ipykernel_14152/1632527333.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#cns_Tab21').click()


In [107]:
df_074600

주요재무정보,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,...,PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주),name,code,year
2017/12\n\t\t\t\t\t\t\t\t\t\n(IFRS연결),"1,973",296,296,369,294,289,5,"2,569",759,"1,810",...,15.00,"6,503",2.54,0,0.00,0.00,"26,288,000",pp,074600,2017
2018/12\n\t\t\t\t\t\t\t\t\t\n(IFRS연결),"2,665",412,412,477,412,405,7,"3,595","1,384","2,211",...,6.49,"8,003",1.25,0,0.00,0.00,"26,288,000",pp,074600,2018
2019/12\n\t\t\t\t\t\t\t\t\t\n(IFRS연결),"2,631",274,274,1,36,26,10,"6,410","3,538","2,872",...,157.36,"8,103",1.93,0,0.00,0.00,"26,288,000",pp,074600,2019
2020/12\n\t\t\t\t\t\t\t\t\t\n(IFRS연결),"5,256",412,412,221,155,258,-104,"7,431","4,405","3,026",...,21.64,"9,070",2.34,0,0.00,0.00,"26,288,000",pp,074600,2020
2021/12\n\t\t\t\t\t\t\t\t\t\n(IFRS연결),"6,241",868,868,840,625,588,38,"8,236","4,438","3,798",...,12.80,"11,705",2.44,150,0.52,6.71,"26,288,000",pp,074600,2021


### 투자지표 TEST

In [12]:
driver = webdriver.Chrome()
driver.get('https://navercomp.wisereport.co.kr/v2/company/c1040001.aspx?cmp_cd=290650&cn=')
time.sleep(2)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
temp = soup.find_all('table')

In [13]:
p = parser.make2d(temp[9])
p

[['항목',
  '2017/12(IFRS별도)',
  '2018/12(IFRS연결)',
  '2019/12(IFRS연결)',
  '2020/12(IFRS연결)',
  '2021/12(IFRS연결)\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t연간컨센서스보기',
  '2022/12(E)(IFRS연결)\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t연간컨센서스닫기',
  '전년대비(YoY)',
  '전년대비(YoY)'],
 ['펼치기\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\tEPS',
  '230',
  '276',
  '331',
  '492',
  '618',
  '458',
  '26',
  '-26'],
 ['당기순이익(지배)＜당기＞', '37.5', '52.1', '72.5', '108.1', '138.1', '104.0', '', ''],
 ['보통주수정평균발행주식수＜당기＞',
  '15,985,602',
  '18,882,396',
  '21,910,860',
  '21,991,707',
  '22,346,244',
  '22,684,891',
  '',
  ''],
 ['우선주수정평균발행주식수＜당기＞', '321,233', '0', '0', '0', '0', '0', '', ''],
 ['펼치기\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\tBPS',
  '939',
  '2,417',
  '2,635',
  '2,642',
  '3,748',
  '3,482',
  '42',
  '-7'],
 ['자본총계(지배)＜당기＞',
  '168.1',
  '524.6',
  '581.8',
  '578.1',
  '850.2',
  '790.0',
  '',
  ''],
 ['보통주수정기말발행주식수(자사주차감)＜당기＞',
  '17,913,00

In [27]:
df = pd.DataFrame(p[23:26], columns = p[0])
df = df.set_index('항목').T
df = df.rename(columns = {'펼치기\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\tPCR' : 'PCR',
                    '보통주수정주가(기말)＜당기＞' : '수정주가',
                    'CPS＜당기＞' : 'CPS'})

In [31]:
df['year'] = [2017, 2018, 2019, 2020, 2021, 0, 0, 0]
df = df.drop(df.index[5:])

In [32]:
df['code'] = '290650'
df

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_16268/2264154378.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['code'] = '290650'


항목,PCR,수정주가,CPS,year,code
2017/12(IFRS별도),,,234,2017,290650
2018/12(IFRS연결),32.16,"6,550",204,2018,290650
2019/12(IFRS연결),29.07,"8,533",294,2019,290650
2020/12(IFRS연결),102.27,"34,500",337,2020,290650
2021/12(IFRS연결)\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t연간컨센서스보기,74.99,"36,600",488,2021,290650


### Selenium 투자지표

In [36]:
name_code

,name,code
0,삼성전자우,005935
1,삼성전자,005930
2,SK하이닉스,000660
3,LG 화학,051915
4,삼성바이오로직스,207940
...,...,...
1722,EMNI,083470
1723,지나인제약,078650
1724,에스디시스템,121890
1725,SNK,950180


In [69]:
name = list(name_code['name'])

In [70]:
code = list(name_code['code'])

In [71]:
name_codes = dict(zip(name, code))

In [72]:
name_codes

{'삼성전자우': '005935',
 '삼성전자': '005930',
 'SK하이닉스': '000660',
 'LG 화학': '051915',
 '삼성바이오로직스': '207940',
 '현대차2우B': '005387',
 '현대차우': '005385',
 '현대차3우B': '005389',
 '삼성SDI': '006400',
 '삼성SDI우': '006405',
 'NAVER': '035420',
 'LG화학': '051910',
 '현대차': '005380',
 '카카오': '035720',
 '기아': '000270',
 '삼성물산우B': '02826K',
 'LG전자우': '066575',
 'LG생활건강우': '051905',
 'SK이노우': '096775',
 '셀트리온': '068270',
 'KB금융': '105560',
 '신한지주': '055550',
 'POSCO홀딩스': '005490',
 'LG전자': '066570',
 'SK우': '03473K',
 '삼성물산': '028260',
 '현대모비스': '012330',
 'SK이노베이션': '096770',
 'LG우': '003555',
 '한국전력': '015760',
 '삼성전기우': '009155',
 'SK그룹': '034730',
 'HMM': '011200',
 '하나금융지주': '086790',
 '아모레퍼시픽우': '090435',
 '두산중공업': '034020',
 'LG생활건강': '051900',
 'LG': '003550',
 'S-oil': '010950',
 'SK텔레콤': '017670',
 '삼성전기': '009150',
 '삼성생명': '032830',
 '삼성에스디에스': '018260',
 'S-Oil우': '010955',
 '우리금융지주': '316140',
 '삼성화재 우선주': '000815',
 '대한항공': '003490',
 '아모레퍼시픽': '090430',
 '대한항공우': '003495',
 'KT&G': '033780',
 '고

In [76]:
len(name_codes)

1727

In [73]:
df_o = []
cnt = 0


for i,j in name_codes.items():
    try:
        driver = webdriver.Chrome()
        driver.get(f'https://navercomp.wisereport.co.kr/v2/company/c1040001.aspx?cmp_cd={j}&cn=')
#         driver.find_element_by_css_selector('#cns_Tab21').click()
        time.sleep(1.5)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        temp = soup.find_all('table')
        p = parser.make2d(temp[9])
        globals()['df_'+str(j)]= pd.DataFrame(p[23:26], columns = p[0])
        globals()['df_'+str(j)]=globals()['df_'+str(j)].set_index('항목').T
        globals()['df_'+str(j)] = globals()['df_'+str(j)].rename(columns = {'펼치기\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\tPCR' : 'PCR',
                    '보통주수정주가(기말)＜당기＞' : '수정주가',
                    'CPS＜당기＞' : 'CPS'})
#         globals()['df_'+str(j)]['name'] = i
        globals()['df_'+str(j)]['code'] = j
        globals()['df_'+str(j)]['year'] = [2017, 2018, 2019, 2020, 2021, 0, 0, 0]   
        globals()['df_'+str(j)]=globals()['df_'+str(j)].drop(globals()['df_'+str(j)].index[5:])


        df_o.append(globals()['df_'+str(j)])

        cnt += 1
        print(cnt)
    
    except:
        pass

df_invest_statements = pd.concat(df_o)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [74]:
df_invest_statements

항목,PCR,수정주가,CPS,code,year
2017/12(IFRS연결),6.25,"50,960","8,151",005935,2017
2018/12(IFRS연결),4.21,"38,700","9,200",005935,2018
2019/12(IFRS연결),8.35,"55,800","6,681",005935,2019
2020/12(IFRS연결),8.43,"81,000","9,611",005935,2020
2021/12(IFRS연결)\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t연간컨센서스보기,8.17,"78,300","9,585",005935,2021
...,...,...,...,...,...
2017/12(IFRS연결),-37.20,"27,300",-734,050540,2017
2018/12(IFRS연결),-23.77,"30,500","-1,283",050540,2018
2019/12(IFRS연결),-5.78,"20,350","-3,522",050540,2019
2020/12(IFRS연결),2.21,"2,820","1,276",050540,2020


In [75]:
df_invest_statements.to_csv('투자지표.csv')

In [ ]:
li = ['108320', '077500', '091090', '004060']

In [ ]:
df_oo = []
cnt = 0

for i in li:
    try:
        driver = webdriver.Chrome()
        driver.get(f'https://navercomp.wisereport.co.kr/v2/company/c1040001.aspx?cmp_cd={i}&cn=')
#         driver.find_element_by_css_selector('#cns_Tab21').click()
        time.sleep(1.5)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        temp = soup.find_all('table')
        p = parser.make2d(temp[9])
        globals()['df_'+str(i)]= pd.DataFrame(p[23:26], columns = p[0])
        globals()['df_'+str(i)]=globals()['df_'+str(i)].set_index('항목').T
        globals()['df_'+str(i)] = globals()['df_'+str(i)].rename(columns = {'펼치기\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\tPCR' : 'PCR',
                    '보통주수정주가(기말)＜당기＞' : '수정주가',
                    'CPS＜당기＞' : 'CPS'})
#         globals()['df_'+str(j)]['name'] = i
        globals()['df_'+str(i)]['code'] = i
        globals()['df_'+str(i)]['year'] = [2017, 2018, 2019, 2020, 2021, 0, 0, 0]   
        globals()['df_'+str(i)]=globals()['df_'+str(i)].drop(globals()['df_'+str(i)].index[5:])


        df_oo.append(globals()['df_'+str(i)])

        cnt += 1
        print(cnt)
    
    except:
        pass

df_rest = pd.concat(df_oo)

In [90]:
df_rest

항목,PCR,수정주가,CPS,code,year
2017/12(IFRS연결),26.47,"52,900","1,999",108320,2017
2018/12(IFRS연결),19.96,"33,600","1,683",108320,2018
2019/12(IFRS연결),31.28,"39,800","1,272",108320,2019
2020/12(IFRS연결),11.41,"58,200","5,100",108320,2020
2021/12(IFRS연결)\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t연간컨센서스보기,6.46,"167,300","25,916",108320,2021
2017/12(IFRS연결),121.68,"9,660",79,077500,2017
2018/12(IFRS연결),14.18,"6,380",450,077500,2018
2019/12(IFRS연결),-7.45,"6,670",-895,077500,2019
2020/12(IFRS연결),45.57,"14,700",323,077500,2020
2021/12(IFRS연결)\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t연간컨센서스보기,33.78,"12,100",358,077500,2021


In [ ]:
df_invest_statements = pd.concat([df_rest, df_invest_statements])
df_invest_statements